In [ ]:
# TODO: Replace these values with what you found in Module 1
diameter = 0.06 # meters
length = 0.12 # meters
max_speed = 44.062 # rad/s

from jetbot import Robot
import time
import numpy as np
from IPython.display import display
import ipywidgets as widgets
import threading

robot = Robot()
radius = diameter/2.0
    
class Robot_Location():
    x = 0.0
    y = 0.0
    theta = 0.0
    t = None
    
    def update_pose(self):
        if self.t is None:
            self.t = time.time()
        else:
            w_R = robot.right_motor.value*max_speed
            w_L = robot.left_motor.value*max_speed

            # TODO: Write the lines for these variables using the equations above
            # Note that you can use np.sin and np.cos for the trigonometric functions
            x_dot = radius/2*(w_R + w_L)*np.cos(np.deg2rad(self.theta))
            y_dot = radius/2*(w_R + w_L)*np.sin(np.deg2rad(self.theta))
            theta_dot = np.rad2deg(radius/length*(w_R - w_L))

            t_new = time.time()
            dt = t_new - self.t
            self.x = x_dot*dt + self.x
            self.y = y_dot*dt + self.y
            self.theta = ((theta_dot*dt + self.theta) + 360) % 360
            self.t = t_new

In [ ]:
x_coords = []
y_coords = []

def update_odometry():
    global robot_enabled
    global x_coords
    global y_coords
    while not stop_event.is_set():
        robot_location.update_pose()
        try:
            x_widget.value = f"X: {robot_location.x:.3f}"
            y_widget.value = f"Y: {robot_location.y:.3f}"
            theta_widget.value = f"\u03B8: {robot_location.theta:.3f}°"
        except Exception as e:
            pass
        if robot_enabled:
            x_coords.append(robot_location.x)
            y_coords.append(robot_location.y)
        time.sleep(0.1)
            

robot_enabled = False
stop_event = threading.Event()

In [ ]:
# Widgets to display coordinates
x_widget = widgets.Label(value="X: ---")
y_widget = widgets.Label(value="Y: ---")
theta_widget = widgets.Label(value="\u03B8: ---")

display(widgets.HBox([x_widget, y_widget, theta_widget]))

robot_location = Robot_Location()
thread = threading.Thread(target=update_odometry, daemon=True)
thread.start()
x_coords = []
y_coords = []

# Add your robot path between the two dashed lines!
robot_enabled = True
# ---------- Start Here ----------
robot.forward(0.2)
time.sleep(3.0)
robot.right(0.1)
time.sleep(1.0)
robot.backward(0.15)
time.sleep(6.0)
robot.set_motors(0.3, 0.2)
time.sleep(2.0)
# ----------- End Here -----------
robot.stop()
robot_enabled = False

stop_event.set()
thread.join()
stop_event.clear()

In [ ]:
import cv2
import numpy as np
from IPython.display import display, Image
from jetbot import bgr8_to_jpeg

# Set logical bounds
min_x = int(min(x_coords)) - 1
max_x = int(max(x_coords)) + 1
min_y = int(min(y_coords)) - 1
max_y = int(max(y_coords)) + 1

# Image canvas
image_width, image_height = 720, 540
image = np.ones((image_height, image_width, 3), dtype=np.uint8) * 255  # white background

# Padding around the plot area (in pixels)
pad = 40
plot_width = image_width - 2 * pad
plot_height = image_height - 2 * pad

# Coordinate transform
def logical_to_image_coords(x, y):
    u = int(pad + (x - min_x) / (max_x - min_x) * plot_width)
    v = int(pad + (max_y - y) / (max_y - min_y) * plot_height)
    return u, v

# Get axis pixel location
axis_x, axis_y = logical_to_image_coords(0, 0)

# Vertical gridlines (extend full height)
for gx in range(min_x, max_x + 1):
    if gx != 0:
        x, _ = logical_to_image_coords(gx, 0)
        cv2.line(image, (x, 0), (x, image_height - 1), (220, 220, 220), 1)
        cv2.putText(image, str(gx), (x - 10, axis_y + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100, 100, 100), 1)

# Horizontal gridlines (extend full width)
for gy in range(min_y, max_y + 1):
    if gy != 0:
        _, y = logical_to_image_coords(0, gy)
        cv2.line(image, (0, y), (image_width - 1, y), (220, 220, 220), 1)
        cv2.putText(image, str(gy), (axis_x + 5, y + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100, 100, 100), 1)

# Draw axes
x0, y0 = logical_to_image_coords(0, min_y)
x1, y1 = logical_to_image_coords(0, max_y)
cv2.line(image, (0, axis_y), (image_width - 1, axis_y), (0, 0, 0), 2)


x0, y0 = logical_to_image_coords(min_x, 0)
x1, y1 = logical_to_image_coords(max_x, 0)
cv2.line(image, (axis_x, 0), (axis_x, image_height - 1), (0, 0, 0), 2)

for i, (x, y) in enumerate(zip(x_coords, y_coords)):
    u, v = logical_to_image_coords(x, y)
    if i == 0:
        color = (0, 255, 0)    # Green
    elif i == len(x_coords) - 1:
        color = (0, 0, 255)    # Red
    else:
        color = (255, 0, 0)    # Blue
    cv2.circle(image, (u, v), 5, color, -1)

# Save and display
cv2.imwrite('robot_path.jpg', image)
display(Image(data=bgr8_to_jpeg(image)))